# Data Visualization

This notebook demonstrates the process of visualizing data stored in Delta Lake using Dash

In [ ]:

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

signal_name = "sensor_data_depth"
df = duckpond.conn.execute(f"""
SELECT signal_name, datetime, data_labels, values
FROM DataLake
WHERE signal_name = $1
""", [signal_name]).df()
display(df)

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.graph_objects as go
import importlib

import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

signal_names = duckpond.conn.sql(f"""
                                     SELECT DISTINCT signal_name
                                     FROM DataLake
                                     WHERE animal = 'oror-002'
                                     """).df()

signal_names = sorted([name for name in signal_names.signal_name.to_list() if (name.startswith("sensor") or name.startswith("logger"))])

app = Dash()

app.layout = [
    dcc.Dropdown(signal_names, signal_names[0], id='dropdown-selection'),    
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = duckpond.get_delta_data(    
        animal_ids="oror-002",
        frequency=10,
        signal_names=value,
    )
    print(dff)
    
    fig = go.Figure()
    for col in dff.columns[1:]:
        fig.add_trace(
            go.Scattergl(
                x=dff['datetime'],
                y=dff[col],
                mode='lines',
                name=col
            )
        )
    return fig

app.run(debug=True, host='0.0.0.0')

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash()

app.layout = [
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')
